# Tutorial 2.1. Preprocessing CBP Water Quality Data

## 2.1.1 Introduction

This notebook is **Step1** for the *Predict Future DO Tutorial Series*.

It preprocesses Chesapeake Bay Program (CBP) water quality data downloaded from the [CBP DataHub](https://datahub.chesapeakebay.net/WaterQuality).

- **Data split:** To avoid timeouts, data is downloaded in three regional groups—Upper, Mid, and Lower Bay.
- **Goal:** Extract surface DO, Temperature, Salinity, Chla, TN, and TP; align by date and station; and save the cleaned dataset for model training.
- **Files:** Raw data (`Upper.csv`, `Mid.csv`, `Lower.csv`) are in the folder `CBP_RawData`.

> If you need help downloading data from the CBP Datahub, refer to *Tutorial 1*.

---

![Model Diagram](CBP_Mainstem_Station_Map.png)


## 2.1.2 Data Loading & Inspection

In [20]:
import pandas as pd
import numpy as np
import os
from IPython.display import display

# ===  Load all three water quality files ===
folder = "CBP_RawData"
dfs = {}
for fname in ["Upper.csv", "Mid.csv", "Lower.csv"]:
    fpath = os.path.join(folder, fname)
    df = pd.read_csv(fpath)
    dfs[fname] = df
    print(f"\n📂 --- Dataset: {fname} ---")
    
    # --- Show column names ---
    print("🧾 Columns:")
    print(df.columns.tolist())

/var/folders/zp/f07tkd011lb08nh330c0ct600000gn/T/ipykernel_49423/1893105342.py:11: DtypeWarning: Columns (18,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fpath)



📂 --- Dataset: Upper.csv ---
🧾 Columns:
['MonitoringStation', 'EventId', 'Cruise', 'Program', 'Project', 'Agency', 'Source', 'Station', 'SampleDate', 'SampleTime', 'TotalDepth', 'UpperPycnocline', 'LowerPycnocline', 'Depth', 'Layer', 'SampleType', 'SampleReplicateType', 'Parameter', 'Qualifier', 'MeasureValue', 'Unit', 'Method', 'Lab', 'Problem', 'PrecisionPC', 'BiasPC', 'Details', 'Latitude', 'Longitude', 'TierLevel']

📂 --- Dataset: Mid.csv ---
🧾 Columns:
['MonitoringStation', 'EventId', 'Cruise', 'Program', 'Project', 'Agency', 'Source', 'Station', 'SampleDate', 'SampleTime', 'TotalDepth', 'UpperPycnocline', 'LowerPycnocline', 'Depth', 'Layer', 'SampleType', 'SampleReplicateType', 'Parameter', 'Qualifier', 'MeasureValue', 'Unit', 'Method', 'Lab', 'Problem', 'PrecisionPC', 'BiasPC', 'Details', 'Latitude', 'Longitude', 'TierLevel']

📂 --- Dataset: Lower.csv ---
🧾 Columns:
['MonitoringStation', 'EventId', 'Cruise', 'Program', 'Project', 'Agency', 'Source', 'Station', 'SampleDate', 'Sa

## 2.1.3 Explore Each Dataset

Check date range, unique parameters, stations, and preview a random day's data at a random station.

In [23]:
for fname, df in dfs.items():
    print(f"\n📂 --- Dataset: {fname} ---")
    try:
        df['SampleDate'] = pd.to_datetime(df['SampleDate'])
    except Exception as e:
        print("⚠️ Couldn't parse dates:", e)

    # Unique sampling dates
    unique_dates = df['SampleDate'].unique()
    print(f"📅 Number of unique sampling dates: {len(unique_dates)}")

    # Date range
    if pd.api.types.is_datetime64_any_dtype(df['SampleDate']):
        print(f"📆 Date range: {df['SampleDate'].min().date()} to {df['SampleDate'].max().date()}")

    # Unique parameters
    unique_params = df['Parameter'].unique()
    print(f"🧪 Parameters measured ({len(unique_params)}): {unique_params}")

    # Unique stations
    if 'Station' in df.columns:
        unique_stations = df['Station'].unique()
        print(f"📍 Number of unique stations: {len(unique_stations)}")
        print(f"📍 Station list:\n{unique_stations}")

    # Show a random station/day
    valid_rows = df[['SampleDate', 'Station']].dropna()
    if not valid_rows.empty:
        random_idx = np.random.choice(valid_rows.index)
        random_date = valid_rows.loc[random_idx, 'SampleDate']
        random_station = valid_rows.loc[random_idx, 'Station']

        subset = df[(df['SampleDate'] == random_date) & (df['Station'] == random_station)]
        print(f"\n🔍 Showing data for Station '{random_station}' on {random_date.date()}:")
        display(subset)


📂 --- Dataset: Upper.csv ---
📅 Number of unique sampling dates: 1342
📆 Date range: 1984-07-11 to 2024-12-04
🧪 Parameters measured (6): ['CHLA' 'DO' 'SALINITY' 'TN' 'TP' 'WTEMP']
📍 Number of unique stations: 10
📍 Station list:
['CB2.1' 'CB2.2' 'CB3.1' 'CB3.2' 'CB3.3C' 'CB3.3E' 'CB3.3W' 'CB4.1C'
 'CB4.1E' 'CB4.1W']

🔍 Showing data for Station 'CB3.3C' on 2001-08-28:


/var/folders/zp/f07tkd011lb08nh330c0ct600000gn/T/ipykernel_49423/2333169566.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SampleDate'] = pd.to_datetime(df['SampleDate'])


MonitoringStation  EventId  Cruise Program Project Agency Source  \
108386            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
108387            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
108389            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
108390            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
108391            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
...                  ...      ...     ...     ...     ...    ...    ...   
148224            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
148225            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
148226            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
148227            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   
148228            CB3.3C    91357  BAY345    TWQM    MAIN  MDDNR  MDDNR   

       Station SampleDate SampleTime  ...   Unit  Method    Lab  Problem  \
108386  CB3.3C 2001-08-28   15:02:00  ...   UG/L    L01   MDHMH      NaN   
108387  CB3.3C 2001-08-28   15:02:00  ...   UG/L    L01   MDHMH      NaN   
108389  CB3.3C 2001-08-28   15:02:00  ...   UG/L    L01   MDHMH      NaN   
108390  CB3.3C 2001-08-28   15:02:00  ...   UG/L    L01   MDHMH      NaN   
108391  CB3.3C 2001-08-28   15:02:00  ...   UG/L    L01   MDHMH      NaN   
...        ...        ...        ...  ...    ...     ...    ...      ...   
148224  CB3.3C 2001-08-28   15:02:00  ...  DEG C    F01     NaN      NaN   
148225  CB3.3C 2001-08-28   15:02:00  ...  DEG C    F01     NaN      NaN   
148226  CB3.3C 2001-08-28   15:02:00  ...  DEG C    F01     NaN      NaN   
148227  CB3.3C 2001-08-28   15:02:00  ...  DEG C    F01     NaN      NaN   
148228  CB3.3C 2001-08-28   15:02:00  ...  DEG C    F01     NaN      NaN   

       PrecisionPC BiasPC Details  Latitude Longitude  TierLevel  
108386         NaN    NaN     NaN  38.99596 -76.35967         T3  
108387         NaN    NaN     NaN  38.99596 -76.35967         T3  
108389         NaN    NaN     NaN  38.99596 -76.35967         T3  
108390         NaN    NaN     NaN  38.99596 -76.35967         T3  
108391         NaN    NaN     NaN  38.99596 -76.35967         T3  
...            ...    ...     ...       ...       ...        ...  
148224         NaN    NaN     NaN  38.99596 -76.35967         T3  
148225         NaN    NaN     NaN  38.99596 -76.35967         T3  
148226         NaN    NaN     NaN  38.99596 -76.35967         T3  
148227         NaN    NaN     NaN  38.99596 -76.35967         T3  
148228         NaN    NaN     NaN  38.99596 -76.35967         T3  

[81 rows x 30 columns]


📂 --- Dataset: Mid.csv ---
📅 Number of unique sampling dates: 1679
📆 Date range: 1984-07-10 to 2024-12-09
🧪 Parameters measured (6): ['CHLA' 'DO' 'SALINITY' 'TN' 'TP' 'WTEMP']
📍 Number of unique stations: 11
📍 Station list:
['CB4.2C' 'CB4.2E' 'CB4.2W' 'CB4.3C' 'CB4.3E' 'CB4.3W' 'CB5.1' 'CB5.2'
 'CB5.3' 'CB5.4' 'CB5.4W']

🔍 Showing data for Station 'CB5.2' on 2003-09-24:


/var/folders/zp/f07tkd011lb08nh330c0ct600000gn/T/ipykernel_49423/2333169566.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SampleDate'] = pd.to_datetime(df['SampleDate'])


MonitoringStation  EventId  Cruise Program Project Agency Source  \
239754             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
239755             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
239756             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
239757             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
239758             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
...                  ...      ...     ...     ...     ...    ...    ...   
280131             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
280132             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
280133             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
280134             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   
280135             CB5.2    55650  BAY387    TWQM    MAIN  MDDNR  MDDNR   

       Station SampleDate SampleTime  ...   Unit  Method    Lab  Problem  \
239754   CB5.2 2003-09-24    9:49:00  ...   UG/L    L01   MDHMH      NaN   
239755   CB5.2 2003-09-24    9:49:00  ...   UG/L    L01   MDHMH      NaN   
239756   CB5.2 2003-09-24    9:49:00  ...   UG/L    L01   MDHMH      NaN   
239757   CB5.2 2003-09-24    9:49:00  ...   UG/L    L01   MDHMH      NaN   
239758   CB5.2 2003-09-24    9:49:00  ...   UG/L    L01   MDHMH      NaN   
...        ...        ...        ...  ...    ...     ...    ...      ...   
280131   CB5.2 2003-09-24    9:49:00  ...  DEG C    F01     NaN      NaN   
280132   CB5.2 2003-09-24    9:49:00  ...  DEG C    F01     NaN      NaN   
280133   CB5.2 2003-09-24    9:49:00  ...  DEG C    F01     NaN      NaN   
280134   CB5.2 2003-09-24    9:49:00  ...  DEG C    F01     NaN      NaN   
280135   CB5.2 2003-09-24    9:49:00  ...  DEG C    F01     NaN      NaN   

       PrecisionPC BiasPC Details  Latitude Longitude  TierLevel  
239754         NaN    NaN     NaN  38.13705 -76.22787         T3  
239755         NaN    NaN     NaN  38.13705 -76.22787         T3  
239756         NaN    NaN     NaN  38.13705 -76.22787         T3  
239757         NaN    NaN     NaN  38.13705 -76.22787         T3  
239758         NaN    NaN     NaN  38.13705 -76.22787         T3  
...            ...    ...     ...       ...       ...        ...  
280131         NaN    NaN     NaN  38.13705 -76.22787         T3  
280132         NaN    NaN     NaN  38.13705 -76.22787         T3  
280133         NaN    NaN     NaN  38.13705 -76.22787         T3  
280134         NaN    NaN     NaN  38.13705 -76.22787         T3  
280135         NaN    NaN     NaN  38.13705 -76.22787         T3  

[75 rows x 30 columns]


📂 --- Dataset: Lower.csv ---
📅 Number of unique sampling dates: 1703
📆 Date range: 1984-07-11 to 2024-12-16


/var/folders/zp/f07tkd011lb08nh330c0ct600000gn/T/ipykernel_49423/2333169566.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SampleDate'] = pd.to_datetime(df['SampleDate'])


🧪 Parameters measured (6): ['CHLA' 'DO' 'SALINITY' 'TN' 'TP' 'WTEMP']
📍 Number of unique stations: 15
📍 Station list:
['CB5.5' 'CB6.1' 'CB6.2' 'CB6.3' 'CB6.4' 'CB7.1' 'CB7.1N' 'CB7.2' 'CB7.2E'
 'CB7.3' 'CB7.3E' 'CB7.4' 'CB7.4N' 'CB8.1' 'CB8.1E']

🔍 Showing data for Station 'CB7.2' on 1999-01-13:


MonitoringStation  EventId  Cruise Program Project Agency Source  \
213603             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
213604             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223055             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223056             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223057             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223058             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223059             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223060             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223061             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223062             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223063             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223064             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223065             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223066             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223067             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223068             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223069             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223070             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223071             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
223072             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233876             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233877             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233878             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233879             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233880             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233881             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233882             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233883             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233884             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233885             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233886             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233887             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233888             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233889             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233890             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233891             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233892             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
233893             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
236182             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
236183             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
237319             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
237320             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247308             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247309             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247310             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247311             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247312             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247313             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247314             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247315             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247316             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247317             CB7.2    85666  BAY292    TWQM    MAIN  VADEQ    ODU   
247318             CB7.2    8566

## 2.1.4 Clean and Filter Data

We filter the dataset to retain only valid records, focus on surface-layer samples, and keep variables relevant to DO prediction.

In [26]:
# Combine all regional datasets
df_wq = pd.concat(dfs.values(), ignore_index=True)

# Ensure dates are parsed
df_wq["SampleDate"] = pd.to_datetime(df_wq["SampleDate"])

# Filter for surface data (Depth = 0.5m)
df_surface = df_wq[df_wq["Depth"] == 0.5].copy()

# Pivot table: One row per station-date, columns are parameters
df_pivot = df_surface.pivot_table(
    index=["Station", "SampleDate"],
    columns="Parameter",
    values="MeasureValue",
    aggfunc="mean"  # Average if duplicates
).reset_index()

# Optional: Remove column hierarchy if created by pivot
df_pivot.columns.name = None

# Preview the final dataset
print("📋 Preview of cleaned water quality data:")
print(df_pivot.head())



📋 Preview of cleaned water quality data:
  Station SampleDate  CHLA   DO  SALINITY      TN     TP  WTEMP
0   CB2.1 1984-07-12  10.7  7.8      0.00  1.8515  0.120   25.9
1   CB2.1 1984-07-25  26.7  7.6      0.00  1.3990  0.086   27.6
2   CB2.1 1984-09-12   6.7  7.1      0.73  1.1870  0.048   22.6
3   CB2.1 1984-09-26  12.0  7.8      1.97  1.0730  0.049   22.0
4   CB2.1 1984-10-10  14.7  8.7      2.89  1.6200  0.046   16.1


## 2.1.5 Export Cleaned Data

Save the cleaned and pivoted dataset for later use in model training.


In [31]:
import os

# Ensure the folder exists
os.makedirs("CleanedData", exist_ok=True)

# Save to the CleanedDataset folder
output_path = os.path.join("CleanedData", "CBP_water_quality_surface.csv")
df_pivot.to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")


✅ Saved: CleanedData/CBP_water_quality_surface.csv
